In [ ]:
"""
Script Functionality:
This script calculates the total, mean, minimum (non-zero), and maximum precipitation for Ghana in 2021 
using Earth Engine and precipitation data from the CHIRPS dataset. 
The results are exported as GeoTIFF files to Google Drive.

Input Parameters:
- Shapefile for Ghana (replace the path to match your local environment).
- CHIRPS dataset for daily precipitation in 2021.

Dependencies:
- Earth Engine Python API: pip install earthengine-api
- GeoPandas: pip install geopandas
- Folium (optional for visualization): pip install folium
- JSON (standard library)

Version Information:
- Version: v1.0
- Change Log: October 1 2024.
"""
import geemap
import ee
import geopandas as gpd

ee.Authenticate()  # Authenticate Earth Engine (requires Google account login)
ee.Initialize(project="lizeshuo")  # Initialize Earth Engine with specified project ID

# Step 1: Read the local shapefile and convert it to Earth Engine object
shp_path = 'E:/ghana/Province/1/1/Ghana_Expanded_Boundaries.shp'
gdf = gpd.read_file(shp_path)
roi = geemap.geopandas_to_ee(gdf)  # Convert shapefile to Earth Engine Geometry object

# Step 2: Access the ESA WorldCover dataset and clip it to the region of interest
dataset = ee.ImageCollection('ESA/WorldCover/v200').first()
dataset_clip = dataset.clip(roi)  # Clip using the ROI defined by the shapefile

# Step 3: Export the clipped WorldCover data to Google Drive
output_drive_folder = "EarthEngineExports"  # Name of the Google Drive folder
output_file_prefix = "LC_Export"  # File name prefix

# Define the export task
task = ee.batch.Export.image.toDrive(
    image=dataset_clip,
    description='Export_WorldCover_Clipped_2021',
    folder=output_drive_folder,  # Name of the folder in Google Drive
    fileNamePrefix=output_file_prefix,
    scale=10,  # Adjust the scale if needed (e.g., 10 for WorldCover resolution)
    region=roi.geometry(),  # Use ROI as the export region
    maxPixels=1e13  # Set the maximum number of pixels to export
)

# Start the task
task.start()

print(f"The clipped WorldCover data has been submitted to the Google Drive folder '{output_drive_folder}', with the file name prefix '{output_file_prefix}'")

In [ ]:
import os
from osgeo import gdal

def merge_tif_files(input_folder, output_folder, output_filename):
    # 检查输出文件夹是否存在，如果不存在则创建
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)
    
    # 获取输入文件夹中的所有TIF文件
    tif_files = [os.path.join(input_folder, f) for f in os.listdir(input_folder) if f.endswith('.tif')]

    # 输出文件路径
    output_file = os.path.join(output_folder, output_filename)

    # 检查是否有TIF文件
    if len(tif_files) == 0:
        print("没有找到TIF文件！")
        return

    # 使用gdal.Warp进行融合
    gdal.Warp(output_file, tif_files, format="GTiff")

    print(f"文件已成功融合并保存到: {output_file}")

# 输入文件夹路径，包含TIF文件
input_folder = "E:/ghana/LC_b/LC21/"
# 输出文件夹路径
output_folder = "E:/ghana/LC_b/LC21/"
# 输出文件名
output_filename = "LC2021.tif"

# 调用合并函数
merge_tif_files(input_folder, output_folder, output_filename)

In [ ]:
import rasterio
from rasterio.warp import calculate_default_transform, reproject, Resampling

# 输入和输出路径
lc_path = r'E:\ghana\LC_b\LC20\LC2020.tif' # 土地覆盖栅格路径
template_path = r'E:\ghana\Raster_Templates\ghana_template\ghana\0.0001.tif' # 模板栅格路径
output_path = r'E:\ghana\LC_b\LC20\LC_0.0001\LC_20_0.0001.tif' # 输出路径

with rasterio.open(lc_path) as lc:
    with rasterio.open(template_path) as template:
         # 计算重采样所需的变换参数
         transform, width, height = calculate_default_transform(
             lc.crs, template.crs, template.width, template.height, *template.bounds)
         # 更新元数据
         kwargs = lc.meta.copy()
         kwargs.update({
             'crs': template.crs,
             'transform': transform,
             'width': width,
             'height': height,
             'nodata': lc.nodata,
             'dtype': lc.dtypes[0] # 确保数据类型与原始栅格一致
             })
         # 创建输出栅格
         with rasterio.open(output_path, 'w', **kwargs) as dst:
             for i in range(1, lc.count + 1):
                 reproject(
                     source=rasterio.band(lc, i),
                     destination=rasterio.band(dst, i),
                     src_transform=lc.transform,
                     src_crs=lc.crs,
                     dst_transform=transform,
                     dst_crs=template.crs,
                     resampling=Resampling.nearest
                     )
                 # 复制颜色映射表
                 if lc.colormap(1) is not None:
                     dst.write_colormap(1, lc.colormap(1))
print(f"重采样完成，输出文件保存至: {output_path}")


In [ ]:
"""
Copyright (c) 2024 Yan Jin (jinyan@njupt.edu.cn) and Yong Ge (gey@lreis.ac.cn)
Editor: Zeshuo Li (1023172917@njupt.edu.cn)
Date: October 1 2024
This code is protected by copyright and may not be copied or distributed without permission.

Script Functionality:
This script merges multiple GeoTIFF (.tif) files from a specified input folder 
into a single output GeoTIFF file. It checks if the output directory exists, 
creates it if it doesn't, and uses the GDAL library to perform the merging operation.

Input Parameters:
- input_folder: Path to the folder containing the .tif files to be merged.
- output_folder: Path to the folder where the merged file will be saved.
- output_filename: Name for the merged output file.

Dependencies:
- GDAL: Ensure GDAL is installed for handling geospatial data.
- NumPy: pip install numpy (if used in extended functionality).
- OS: Standard library for file and directory operations.
- Re: Standard library for regular expressions (if used in extended functionality).

Version Information:
- Version: v1.0
- Change Log: October 1 2024.
"""

import os
from osgeo import gdal  # Import the GDAL library for working with geospatial data
import numpy as np
import re

def merge_tif_files(input_folder, output_folder, output_filename):
    # Check if the output folder exists; if not, create it
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)
    
    # Get all TIF files from the input folder
    tif_files = [os.path.join(input_folder, f) for f in os.listdir(input_folder) if f.endswith('.tif')]  # List all .tif files in the directory

    # Define the output file path
    output_file = os.path.join(output_folder, output_filename)  # Set the full path for the output file

    # Check if there are any TIF files in the folder
    if len(tif_files) == 0:
        print("No TIF files found!")  # Print message if no .tif files are found in the folder
        return

    # Use gdal.Warp to merge the TIF files
    gdal.Warp(output_file, tif_files, format="GTiff")  # Use GDAL Warp to merge the input .tif files into a single output file in GTiff format

    print(f"Files successfully merged and saved to: {output_file}")  # Confirmation message when the process is complete

# Input folder path containing the TIF files
input_folder = "E:/ghana/LC_b/LC_21_01"  # Specify the folder where the .tif files are located
# Output folder path
output_folder = "E:/ghana/LC_b/LC_21_01/LC_21_01"  # Specify the folder where the merged file should be saved
# Output file name
output_filename = "LC21_01.tif"  # Set the desired name for the output file

# Call the function to merge the TIF files
merge_tif_files(input_folder, output_folder, output_filename)  # Run the function with the specified input and output paths


In [ ]:
"""
Copyright (c) 2024 Yan Jin (jinyan@njupt.edu.cn) and Yong Ge (gey@lreis.ac.cn)
Editor: Zeshuo Li (1023172917@njupt.edu.cn)
Date: October 1 2024
This code is protected by copyright and may not be copied or distributed without permission.

Script Functionality:
This script extracts specific land cover types from a GeoTIFF input file,
creating separate GeoTIFF files for each land cover type defined in the land_cover_types dictionary.
It ensures that filenames are sanitized to avoid invalid characters.

Input Parameters:
- input_tif: Path to the input GeoTIFF file containing land cover data.
- output_folder: Path to the folder where the extracted land cover files will be saved.

Dependencies:
- GDAL: Required for reading and writing geospatial data.
- NumPy: pip install numpy (for array manipulation).
- OS: Standard library for file and directory operations.
- Re: Standard library for regular expressions (used for filename sanitization).

Version Information:
- Version: v1.0
- Change Log: October 1 2024.
"""

import os
from osgeo import gdal  # Import GDAL library to work with geospatial data
import numpy as np  # Import NumPy for array manipulation
import re  # Import regular expressions for filename sanitization

# Define land cover types and their attributes
land_cover_types = {
    10: "Tree cover",
    20: "Shrubland",
    30: "Grassland",
    40: "Cropland",
    50: "Built-up",
    60: "Bare / sparse vegetation",
    70: "Snow and ice",
    80: "Permanent water bodies",
    90: "Herbaceous wetland",
    95: "Mangroves",
    100: "Moss and lichen"
}

def sanitize_filename(name):
    """
    Replace invalid characters in the filename with an underscore.
    Only keep letters, numbers, underscores, and dashes.
    """
    # Use regular expressions to replace illegal characters with underscores
    return re.sub(r'[^\w\-]', '_', name)

def extract_land_cover(input_tif, output_folder):
    # Check if the output folder exists; if not, create it
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    # Open the TIF file
    dataset = gdal.Open(input_tif)
    if dataset is None:
        print(f"Unable to open input TIF file: {input_tif}")
        return
    
    band = dataset.GetRasterBand(1)  # Read the first raster band
    array = band.ReadAsArray()  # Convert the raster band to a NumPy array

    # Loop through each land cover type
    for value, description in land_cover_types.items():
        # Create a new array, keeping the current type's values and setting others to 0
        new_array = np.where(array == value, array, 0)

        # Sanitize the filename to ensure it's valid
        sanitized_description = sanitize_filename(description)

        # Define the output filename
        output_tif = os.path.join(output_folder, f"{sanitized_description}.tif")

        # Get the GDAL driver
        driver = gdal.GetDriverByName("GTiff")
        if driver is None:
            print("Failed to load GDAL driver; cannot get GTiff driver.")
            return

        # Create the output TIF file
        out_dataset = driver.Create(output_tif, dataset.RasterXSize, dataset.RasterYSize, 1, band.DataType)
        if out_dataset is None:
            print(f"Unable to create output file: {output_tif}")
            continue
        
        out_band = out_dataset.GetRasterBand(1)  # Get the output file's raster band

        # Write the new array to the output band
        out_band.WriteArray(new_array)
        out_band.FlushCache()  # Ensure the data is written to disk

        # Set geotransformation and projection information
        out_dataset.SetGeoTransform(dataset.GetGeoTransform())
        out_dataset.SetProjection(dataset.GetProjection())

        print(f"{description} extraction completed, file saved to: {output_tif}")

    # Close the dataset
    dataset = None

# Call the extraction function
input_tif = "E:/ghana/LC_b/LC_21_01/LC_21_01/LC21_01.tif"  # Path to the input TIF file
output_folder = "E:/ghana/LC_b/LC_21_01/LC_21_01/extract"  # Path to the output folder

extract_land_cover(input_tif, output_folder)


In [ ]:
"""
Copyright (c) 2024 Yan Jin (jinyan@njupt.edu.cn) and Yong Ge (gey@lreis.ac.cn)
Editor: Zeshuo Li (1023172917@njupt.edu.cn)
Date: October 1 2024
This code is protected by copyright and may not be copied or distributed without permission.


Script Functionality:
This script processes multiple GeoTIFF files with a 0.0001-degree resolution,
calculating the percentage of various land cover types within each pixel of a specified 
0.005-degree resolution template. The results are saved as separate GeoTIFF files for each land cover type.

Input Parameters:
- input_folder: Path to the folder containing the 0.0001-degree resolution TIF files.
- template_tif: Path to the 0.005-degree resolution template TIF file.
- output_folder: Path to the folder where the resulting TIF files will be saved.

Dependencies:
- GDAL: Required for reading and writing geospatial data (ensure GDAL is installed).
- NumPy: pip install numpy (for array manipulation).
- OS: Standard library for file and directory operations.

Version Information:
- Version: v1.0
- Change Log: October 1 2024.
"""


import os
import numpy as np
from osgeo import gdal

# Define each land cover type and its value
land_cover_types = {
    10: "Tree_cover",
    20: "Shrubland",
    30: "Grassland",
    40: "Cropland",
    50: "Built_up",
    60: "Bare_sparse_vegetation",
    70: "Snow_and_ice",
    80: "Permanent_water_bodies",
    90: "Herbaceous_wetland",
    95: "Mangroves",
    100: "Moss_and_lichen"
}

def resample_land_cover(input_tif, template_tif, output_folder):
    """
    Calculate the percentage of each land cover type within the 0.005-degree resolution template pixel
    based on a 0.0001-degree resolution input, and generate a new TIF image for each land cover type.

    :param input_tif: Input TIF file with 0.0001-degree land cover data
    :param template_tif: 0.005-degree resolution template TIF file
    :param output_folder: Output folder to save the resulting TIF files
    """
    # Read the 0.0001-degree land cover TIF file
    src_ds = gdal.Open(input_tif)
    if src_ds is None:
        print(f"Unable to open input TIF file: {input_tif}")
        return
    src_band = src_ds.GetRasterBand(1)
    src_array = src_band.ReadAsArray()  # Read the input raster data as a NumPy array
    
    # Read the 0.005-degree resolution template TIF
    template_ds = gdal.Open(template_tif)
    if template_ds is None:
        print(f"Unable to open template TIF file: {template_tif}")
        return
    template_band = template_ds.GetRasterBand(1)  # Read the template raster band
    
    # Get georeferencing information from the input and template
    src_transform = src_ds.GetGeoTransform()
    template_transform = template_ds.GetGeoTransform()

    # Get the resolution of the input and template files
    src_pixel_width = src_transform[1]
    template_pixel_width = template_transform[1]

    # Calculate how many input TIF pixels fit into one template pixel
    ratio = int(template_pixel_width / src_pixel_width)

    # Get the size (number of pixels) of the template raster
    template_x_size = template_ds.RasterXSize
    template_y_size = template_ds.RasterYSize

    # Create the output folder if it doesn't exist
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    # Iterate over each land cover type
    for value, description in land_cover_types.items():
        # Create a new output TIF file for each land cover type
        output_tif = os.path.join(output_folder, f"{description}.tif")

        # Create the output TIF file
        driver = gdal.GetDriverByName("GTiff")
        out_ds = driver.Create(output_tif, template_x_size, template_y_size, 1, gdal.GDT_Float32)
        out_ds.SetGeoTransform(template_transform)
        out_ds.SetProjection(template_ds.GetProjection())
        out_band = out_ds.GetRasterBand(1)

        # Initialize the output array
        out_array = np.zeros((template_y_size, template_x_size), dtype=np.float32)

        # Loop through each pixel of the template and calculate the proportion of the target land cover type
        for i in range(template_y_size):
            for j in range(template_x_size):
                # Calculate the corresponding area in the 0.0001-degree resolution raster
                src_i_start = int(i * ratio)
                src_i_end = int(src_i_start + ratio)
                src_j_start = int(j * ratio)
                src_j_end = int(src_j_start + ratio)

                # Ensure we don't exceed the bounds of the source array
                if src_i_end > src_array.shape[0]:
                    src_i_end = src_array.shape[0]
                if src_j_end > src_array.shape[1]:
                    src_j_end = src_array.shape[1]

                # Extract the region corresponding to the current template pixel
                sub_array = src_array[src_i_start:src_i_end, src_j_start:src_j_end]

                # Calculate the percentage of the target land cover type within this region
                target_count = np.sum(sub_array == value)
                total_count = sub_array.size

                # Calculate the proportion and assign it to the output array
                if total_count > 0:
                    out_array[i, j] = target_count / total_count
                else:
                    out_array[i, j] = 0

        # Write the output array to the output TIF file
        out_band.WriteArray(out_array)
        out_band.FlushCache()

        print(f"TIF file for {description} has been saved to: {output_tif}")

    # Close the datasets
    src_ds = None
    template_ds = None

def process_folder(input_folder, template_tif, output_folder):
    """
    Process each TIF file in the folder, calculate the proportion of each land cover type
    within the 0.005-degree template, and save the results.

    :param input_folder: Folder containing the 0.0001-degree resolution TIF files
    :param template_tif: 0.005-degree resolution template TIF file
    :param output_folder: Output folder for the resulting files
    """
    # Loop through all TIF files in the input folder
    for filename in os.listdir(input_folder):
        if filename.endswith(".tif"):
            input_tif = os.path.join(input_folder, filename)
            file_output_folder = os.path.join(output_folder, filename.replace(".tif", ""))  # Create a subfolder for each file

            print(f"Processing: {input_tif}")
            
            # Call the resample_land_cover function to perform the calculations
            resample_land_cover(input_tif, template_tif, file_output_folder)

# Example usage
input_folder = "E:/ghana/LC_b/LC/LC21/LC_0.0001"  # Folder containing the 0.0001-degree resolution TIF files
template_tif = "E:/ghana/Raster_Templates/ghana_template/0.005.tif"  # Path to the template TIF file
output_folder = "E:/ghana/LC_b/LC/LC21/LC_21_0.005"  # Path to the output folder

process_folder(input_folder, template_tif, output_folder)


In [ ]:
"""
Copyright (c) 2024 Yan Jin (jinyan@njupt.edu.cn) and Yong Ge (gey@lreis.ac.cn)
Editor: Zeshuo Li (1023172917@njupt.edu.cn)
Date: October 1 2024
This code is protected by copyright and may not be copied or distributed without permission.

Script Functionality:
This script processes data at a 0.005-degree scale, aggregating it into a 0.05-degree scale. 

Input Parameters:

Landcover Data: The land cover data with a local resolution of 0.005 degrees.

Dependencies:
rasterio: For reading and writing geospatial raster data, install with pip install rasterio.
NumPy: For efficient array computations, install with pip install numpy.
os: For file and directory operations.


Version Information:
- Version: v1.0
- Change Log: November 3 2024.
"""


import rasterio
import numpy as np
import os

# Set input and output folder paths
input_folder = 'E:/ghana/LC_b/LC/LC20/LC_20_0.005/LC_20_0.005'  # Replace with your input folder path
output_folder = 'E:/ghana/LC_b/LC/LC20/LC_20_0.05'  # Replace with your output folder path

# Ensure the output folder exists
os.makedirs(output_folder, exist_ok=True)

def aggregate_raster(file_path, output_path):
    """
    Aggregate raster data to a lower resolution.

    Parameters:
        file_path (str): Path to the input .tif file.
        output_path (str): Path to the output aggregated file.
    Returns:
        None
    """
    with rasterio.open(file_path) as src:
        source_data = src.read(1)  # Read the first band
        profile = src.profile  # Save metadata

    # Calculate the new number of rows and columns
    new_height = source_data.shape[0] // 10
    new_width = source_data.shape[1] // 10

    # Create an empty array for the aggregated data, initialized to NaN
    aggregated_data = np.empty((new_height, new_width), dtype=source_data.dtype)
    aggregated_data[:] = np.nan

    # Iterate over each 10x10 window and calculate the mean
    for i in range(new_height):
        for j in range(new_width):
            window = source_data[i*10:(i+1)*10, j*10:(j+1)*10]
            if np.count_nonzero(~np.isnan(window)) > 0:  # Check if there are valid values
                aggregated_data[i, j] = np.nanmean(window)  # Compute the mean

    # Update metadata to fit the new dimensions
    profile.update(
        height=new_height,
        width=new_width,
        transform=rasterio.Affine(profile['transform'].a * 10, profile['transform'].b, profile['transform'].c,
                                  profile['transform'].d, profile['transform'].e * 10, profile['transform'].f),
        dtype=aggregated_data.dtype
    )

    # Write the aggregated data to a new file
    with rasterio.open(output_path, 'w', **profile) as dst:
        dst.write(aggregated_data, 1)

# Iterate over all .tif files in the input folder and apply the aggregation function
for filename in os.listdir(input_folder):
    if filename.endswith('.tif'):
        source_path = os.path.join(input_folder, filename)
        output_path = os.path.join(output_folder, filename)
        aggregate_raster(source_path, output_path)

print("Aggregation complete!")

# End of the script